# [실습] OpenAI API를 이용하여 뉴스 기사 정리 봇 만들기

OpenAI Chat API를 이용해 뉴스 검색 결과를 정리하는 프로그램을 만들어 보겠습니다.


네이버의 뉴스 검색 API를 사용하겠습니다.   
(https://developers.naver.com/apps/#/register?defaultScope=search)

In [4]:
!pip install openai tiktoken python-dotenv --upgrade

  Using cached openai-1.99.9-py3-none-any.whl (786 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.25.1
    Uninstalling openai-1.25.1:
      Successfully uninstalled openai-1.25.1



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# os의 환경 변수에 API 키 복사 붙여넣기
import openai
import os

# .env 파일 불러오기
from dotenv import load_dotenv
load_dotenv()

# OPENAI API KEY 설정
import os
api_key = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI()


assert len(os.environ['OPENAI_API_KEY']) > 0, "OPENAI_API_KEY가 환경 변수에 설정되어 있지 않습니다. API 키를 설정해주세요."

# API 키가 설정되어 있다면, 이 지점 이후의 코드가 실행됩니다.
print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

## 1. LLM으로 웹 크롤링 코드 구현 프롬프트 만들기

네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성하겠습니다.   
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 전달하는 함수를 요청합니다.

**네이버 뉴스 API를 사용한다는 내용을 명시하면 정확도가 올라갑니다.**

In [3]:
prompt = '''
네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성해 주세요.
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를 아래 포맷의 문자열로 변환하는 함수 get_news(query) 를 만들어 주세요.
---
제목: (뉴스 제목)
URL: (뉴스 링크- 있으면)
내용: (뉴스 내용)
---

결과는 result에 저장하고, 출력하세요.
---




# 검색어
query = '생성형 AI'


# 검색 결과를 return하는 함수
def get_news(query):


'''



In [4]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [{'role':'system', 'content':'당신은 파이썬 코딩의 전문가입니다. 설명 없이 코드만 출력하세요.'},
          {'role':'user', 'content': prompt}],

    temperature =  0,
    max_tokens = 2000

)
print(response.choices[0].message.content)

```python
import requests

def get_news(query):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=30&sort=sim"
    headers = {
        'X-Naver-Client-Id': 'YOUR_CLIENT_ID',
        'X-Naver-Client-Secret': 'YOUR_CLIENT_SECRET'
    }
    
    response = requests.get(url, headers=headers)
    news_items = response.json().get('items', [])
    
    result = ""
    for item in news_items:
        title = item['title']
        link = item['link']
        description = item['description']
        
        result += f"제목: {title}\n"
        result += f"URL: {link}\n"
        result += f"내용: {description}\n"
        result += "---\n"
    
    return result

# 검색어
query = '생성형 AI'
result = get_news(query)
print(result)
```


In [ ]:
# VS CODE에서 위 결과가 truncated 되는 경우 scrollable element를 클릭하여 풀 코드 확인

아래의 API 키를 사용합니다.
(생성된 코드에 값을 추가하세요)

In [ ]:
# 네이버 API 키: 개발자 센터에서 발급 가능

headers = {
    'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
    'X-Naver-Client-Secret': 'BvqX8mNtHu'
}

아래에 전체 코드를 넣고 실행합니다.

In [10]:
import requests

def get_news(query):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=30&sort=sim"
    headers = {
        'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
        'X-Naver-Client-Secret': 'BvqX8mNtHu'
    }
    
    response = requests.get(url, headers=headers)
    news_items = response.json().get('items', [])
    
    result = ""
    for item in news_items:
        title = item['title']
        link = item['link']
        description = item['description']
        
        result += f"제목: {title}\n"
        result += f"URL: {link}\n"
        result += f"내용: {description}\n"
        result += "---\n"
    
    print(result)
    return result

# 검색어
query = '생성형 AI'
get_news(query)


제목: 반도체·<b>생성형AI</b> 등 AI 밸류체인, &quot;해외 진출 원동력&quot;
URL: https://n.news.naver.com/mnews/article/003/0012909883?sid=101
내용: AI 시대에 한국이 가진 강점으로는 반도체, <b>생성형 AI</b>, 데이터센터, 컴퓨팅 인프라 등 전체 밸류체인을 갖춘 점이 꼽혔다. 윤상두 네이버클라우드 AI랩 소장은 &quot;소버린 AI 경험을 바탕으로 새로운 사업 기회를 창출하고... 
---
제목: &quot;<b>AI</b>은행원과 상담해요&quot;…신한은행, <b>AI</b>브랜치 열어
URL: https://n.news.naver.com/mnews/article/011/0004416396?sid=101
내용: AI 브랜치에서 만날 수 있는 AI 은행원에는 LLM이 반영된 <b>생성형 AI</b>가 적용돼 있다. 고객들은 기존 디지털데스크와는 달리 AI 은행원과 대화를 통해 자연스럽게 상담하고 업무를 처리할 수 있으며 시니어 고객 등... 
---
제목: KT, <b>AI</b>로 제작한 V컬러링 영상 도입
URL: https://n.news.naver.com/mnews/article/009/0005398425?sid=105
내용: KT는 AI로 영상을 분석하고 콘텐츠를 생성할 수 있는 KT 자체 솔루션인 매직플랫폼으로 <b>생성형 AI</b> 기반의 미디어 콘텐츠 서비스를 제공해 왔다. 이번에는 이를 V컬러링 서비스로 확대 적용한다. 특히 중국 최대... 
---
제목: <b>AI</b>검색 시대 열리자 언론사들은 바싹 긴장 중
URL: https://n.news.naver.com/mnews/article/053/0000046854?sid=104
내용: 퍼플렉시티는 자사의 웹페이지에 남긴 장문의 글에서 &quot;<b>생성형 AI</b>기업에 대한 미디어 기업의 소송이 약 30건에 달한다&quot;며 &quot;이번 소송은 근시안적이고 패배적인 미디어와 기술 기업 사이의

"제목: 반도체·<b>생성형AI</b> 등 AI 밸류체인, &quot;해외 진출 원동력&quot;\nURL: https://n.news.naver.com/mnews/article/003/0012909883?sid=101\n내용: AI 시대에 한국이 가진 강점으로는 반도체, <b>생성형 AI</b>, 데이터센터, 컴퓨팅 인프라 등 전체 밸류체인을 갖춘 점이 꼽혔다. 윤상두 네이버클라우드 AI랩 소장은 &quot;소버린 AI 경험을 바탕으로 새로운 사업 기회를 창출하고... \n---\n제목: &quot;<b>AI</b>은행원과 상담해요&quot;…신한은행, <b>AI</b>브랜치 열어\nURL: https://n.news.naver.com/mnews/article/011/0004416396?sid=101\n내용: AI 브랜치에서 만날 수 있는 AI 은행원에는 LLM이 반영된 <b>생성형 AI</b>가 적용돼 있다. 고객들은 기존 디지털데스크와는 달리 AI 은행원과 대화를 통해 자연스럽게 상담하고 업무를 처리할 수 있으며 시니어 고객 등... \n---\n제목: KT, <b>AI</b>로 제작한 V컬러링 영상 도입\nURL: https://n.news.naver.com/mnews/article/009/0005398425?sid=105\n내용: KT는 AI로 영상을 분석하고 콘텐츠를 생성할 수 있는 KT 자체 솔루션인 매직플랫폼으로 <b>생성형 AI</b> 기반의 미디어 콘텐츠 서비스를 제공해 왔다. 이번에는 이를 V컬러링 서비스로 확대 적용한다. 특히 중국 최대... \n---\n제목: <b>AI</b>검색 시대 열리자 언론사들은 바싹 긴장 중\nURL: https://n.news.naver.com/mnews/article/053/0000046854?sid=104\n내용: 퍼플렉시티는 자사의 웹페이지에 남긴 장문의 글에서 &quot;<b>생성형 AI</b>기업에 대한 미디어 기업의 소송이 약 30건에 달한다&quot;며 &quot;이번 소송은 근시안적이고 패배적인

아래는 예시 코드입니다.

In [11]:
import requests

def get_news(query):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=30&sort=sim"
    headers = {
    'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
    'X-Naver-Client-Secret': 'BvqX8mNtHu'

    }

    response = requests.get(url, headers=headers)
    news_json = response.json()

    result = []
    for item in news_json['items']:
        title = item['title']
        link = item['link']
        description = item['description']

        result.append(f"제목: {title}\nURL: {link}\n내용: {description}\n---")

    return "\n".join(result)

# 검색어
query = '생성형 AI'
result = get_news(query)
print(result)

제목: 반도체·<b>생성형AI</b> 등 AI 밸류체인, &quot;해외 진출 원동력&quot;
URL: https://n.news.naver.com/mnews/article/003/0012909883?sid=101
내용: AI 시대에 한국이 가진 강점으로는 반도체, <b>생성형 AI</b>, 데이터센터, 컴퓨팅 인프라 등 전체 밸류체인을 갖춘 점이 꼽혔다. 윤상두 네이버클라우드 AI랩 소장은 &quot;소버린 AI 경험을 바탕으로 새로운 사업 기회를 창출하고... 
---
제목: &quot;<b>AI</b>은행원과 상담해요&quot;…신한은행, <b>AI</b>브랜치 열어
URL: https://n.news.naver.com/mnews/article/011/0004416396?sid=101
내용: AI 브랜치에서 만날 수 있는 AI 은행원에는 LLM이 반영된 <b>생성형 AI</b>가 적용돼 있다. 고객들은 기존 디지털데스크와는 달리 AI 은행원과 대화를 통해 자연스럽게 상담하고 업무를 처리할 수 있으며 시니어 고객 등... 
---
제목: KT, <b>AI</b>로 제작한 V컬러링 영상 도입
URL: https://n.news.naver.com/mnews/article/009/0005398425?sid=105
내용: KT는 AI로 영상을 분석하고 콘텐츠를 생성할 수 있는 KT 자체 솔루션인 매직플랫폼으로 <b>생성형 AI</b> 기반의 미디어 콘텐츠 서비스를 제공해 왔다. 이번에는 이를 V컬러링 서비스로 확대 적용한다. 특히 중국 최대... 
---
제목: <b>AI</b>검색 시대 열리자 언론사들은 바싹 긴장 중
URL: https://n.news.naver.com/mnews/article/053/0000046854?sid=104
내용: 퍼플렉시티는 자사의 웹페이지에 남긴 장문의 글에서 &quot;<b>생성형 AI</b>기업에 대한 미디어 기업의 소송이 약 30건에 달한다&quot;며 &quot;이번 소송은 근시안적이고 패배적인 미디어와 기술 기업 사이의

## 2. OpenAI chat을 이용하여 요약하기

키워드에 대한 뉴스 검색 결과를 요약합니다.   
위 결과인 `result` 데이터를 입력으로 넣은 뒤, 해당 결과에 대한 요약을 요청합니다.

In [12]:
def news_bot(messages):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages = messages,
        temperature = 0.1,
        max_tokens= 2048,
    )

    return response.choices[0].message.content
    # 간편한 출력을 위해 content만 추출

챗봇에 전달할 메시지는 두 개로 나눠집니다.   
위에서 얻은 get_html(query)와, 사용자가 제시할 명령입니다.   

In [15]:
query = '삼성 라이온즈'

result = get_news(query)

response = news_bot([
    {
        # f-string: f"""{변수명}"""
        "role": "system",
        "content": f"""{query}에 대한 뉴스 검색 결과가 주어집니다.
{query}에 대한 뉴스를 요약하세요.
관련 링크를 추가하세요.
---
"""
    },
    {
        "role": "user",
        "content": result
    }
])

print(response)

삼성 라이온즈에 대한 최근 뉴스는 다음과 같습니다:

1. **FA 시장과 스토브리그**: 삼성 라이온즈는 FA 시장에서 핵심 선수들을 놓치며 불펜 보강에 어려움을 겪고 있습니다. 팀은 유망주들의 성장을 대안으로 삼고 있으며, 불펜 보강을 위한 선택지가 줄어들고 있는 상황입니다. (관련 링크: [스포츠뉴스](https://m.sports.naver.com/kbaseball/article/657/0000031936))

2. **마무리 캠프**: 삼성 라이온즈는 일본 오키나와에서 마무리 훈련을 진행하고 있으며, 선수들은 22일까지 훈련을 이어갈 예정입니다. (관련 링크: [스포츠뉴스](https://m.sports.naver.com/kbaseball/article/088/0000915348))

3. **신인 선수 입단식**: 삼성은 2025년 신인 선수 15명의 입단식을 진행했습니다. 이들은 경산과 대구 삼성 라이온즈 파크를 방문한 후 입단식을 가졌습니다. (관련 링크: [대구일보](https://www.idaegu.com/news/articleView.html?idxno=619558))

4. **호주 리그 파견**: 삼성 라이온즈는 유망주들을 호주야구리그에 파견하여 경험을 쌓게 하고 있습니다. (관련 링크: [스포츠뉴스](https://m.sports.naver.com/kbaseball/article/003/0012883548))

5. **팬 사인회**: 삼성 라이온즈의 구자욱 선수가 팬 사인회에 참석하여 팬들과 소통하는 시간을 가졌습니다. (관련 링크: [메트로서울](http://www.metroseoul.co.kr/article/20241113500505))

이 외에도 삼성 라이온즈는 스토브리그에서의 조용한 행보와 불펜 보강의 필요성에 대한 논의가 이어지고 있습니다.


In [17]:
# 토큰 수 체크
import tiktoken

tokenizer  = tiktoken.encoding_for_model('gpt-4o')

print("문자 수:", len(result))
print("토큰 수:", len(tokenizer.encode(result)))
print(result) #result


문자 수: 7557
토큰 수: 4352
제목: tvN 유퀴즈는 왜 '기아' '<b>삼성</b>' 야구팀 이름도 말 못할까
URL: https://n.news.naver.com/mnews/article/006/0000127023?sid=102
내용: 마찬가지로 결승 상대였던 <b>삼성라이온즈</b>를 명시할 때 '삼'만 쓰고 '성'은 별모양으로 처리했다. 온라인 커뮤니티에는 '기별 타이거즈', '기타이거즈' 등으로 꼬집으며 &quot;팀 이름까지 가리네&quot;라는 글들이 올라왔다.... 
---
제목: [삼성, 여기는 오키나와] ① '늦가을에 흘리는 구슬땀' <b>삼성 라이온즈</b>의...
URL: https://m.sports.naver.com/kbaseball/article/088/0000915348
내용: 이번 시즌 준우승을 차지한 <b>삼성 라이온즈</b>는 5일 일본 오키나와 온나손에 둥지를 틀었다. 삼성은 22일까지 마무리 훈련을 진행할 예정이다. 온나손의 아카마 구장을 찾아 구슬땀을 흘리는 선수들의 이야기를 담는다. ①... 
---
제목: 스토브리그 잠잠한 <b>삼성</b>…무소식이 희소식일까
URL: https://www.idaegu.co.kr/news/articleView.html?idxno=481706
내용: 프로야구 <b>삼성 라이온즈</b>가 이대로 스토브리그를 조용히 마감할까. 지난 6일 FA시장이 본격적으로 시작되며 전력 보강을 위해 10개 구단의 머리 싸움이 펼쳐지는 스토브리그가 막을 올렸다. 당초 삼성은 집토끼 단속은... 
---
제목: 호주에서 열린 '<b>삼성</b>-NC' 24시즌 신인 선발 맞대결, 김대호·원종해 승자...
URL: https://m.sports.naver.com/kbaseball/article/241/0003395159
내용: <b>삼성 라이온즈</b>의 김대호와 NC 다이노스의 원종해가 각각 브리즈번 밴디트와 퍼스 히트 유니폼을 입고 선발 마운드에 올랐다. 두 선수 모두 2024시즌 신인 선수들이다

다양한 명령을 넣어서 형식과 스타일을 바꿀 수 있습니다.

In [18]:
query = '삼성 라이온즈'

result  = get_news(query)

result = news_bot([
    {
        "role": "user",
        "content": result
    },
    {
        "role": "user",
        "content": f"""
---
위 전체 내용을 종합하여 {query}의 동향과 미래에 대해 논쟁하는 두 사람의 대화 내용을 만들어줘.
한 명은 전문적인 말투, 한 명은 어리숙한 말투를 사용하고, 형식은 아래와 같아.
---
A:(A의 대화)
B:(B의 대화)
        """
    }
])

print(result)

A: 최근 삼성 라이온즈의 스토브리그에서의 조용한 행보는 다소 우려스럽습니다. FA 시장에서 핵심 자원들을 놓친 상황에서 불펜 보강이 시급한데, 선택지가 줄어드는 모습이 보입니다. 특히, 장현식 영입에 실패한 것은 큰 타격으로 작용할 수 있습니다.

B: 아, 맞아요! 삼성 라이온즈가 요즘 좀 조용한 것 같아요. 불펜 보강이 필요하다고 하는데, 왜 그렇게 안 되는 걸까요? 선수들이 FA 시장에서 다 떠나버리면 어떻게 해요?

A: 그렇습니다. 삼성은 이번 시즌 준우승을 차지했지만, 불펜에서의 약점이 드러났습니다. 특히 역전패가 많았던 점을 고려하면, 마운드의 안정성이 절실합니다. 내부 유망주들의 성장이 대안으로 떠오르고 있지만, 경험이 부족한 선수들로는 한계가 있을 것입니다.

B: 아하, 그래서 신인 선수들이 많이 들어온다고 하더라고요. 그 친구들이 잘할 수 있을까요? 너무 기대가 되기도 하고, 걱정도 되네요.

A: 신인 선수들은 잠재력이 있지만, 프로에서의 경험이 부족하기 때문에 즉각적인 전력 보강에는 한계가 있습니다. 삼성은 이종욱 코치를 영입했지만, 코치진의 변화만으로는 부족할 수 있습니다. 따라서 외부 FA 영입이 필수적입니다.

B: 음, 그러면 삼성 라이온즈가 앞으로 어떻게 될까요? 또 다른 좋은 선수를 영입할 수 있을까요?

A: 현재로서는 불확실성이 큽니다. 삼성의 선택지가 줄어드는 가운데, 내부 육성에 의존할 수밖에 없는 상황입니다. 하지만 만약 적절한 FA 선수를 영입한다면, 다음 시즌에도 경쟁력을 유지할 수 있을 것입니다. 결국, 구단의 전략과 결정이 중요한 시점입니다.

B: 아, 그러니까 삼성 라이온즈가 잘 되려면 좋은 결정을 해야 한다는 거군요! 응원해야겠어요!


## 참고) 스트리밍
ChatGPT처럼 글자가 한 글자씩 순서대로 쭉 출력되도록 하면 어떨까요?   
이와 같은 작업을 스트리밍(Streaming)이라고 하며, 옵션을 추가하여 구현할 수 있습니다.

In [19]:
def news_bot_stream(messages):

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages,
        temperature = 0.1,
        max_tokens = 2048,
        stream=True # 스트리밍
    )

    return response
    # content가 아닌 전체를 return함

In [21]:
query = '거대 언어 모델'
result = get_news(query)

result = news_bot_stream([
    {
        "role": "user",
        "content": result
    },
    {
        "role": "user",
        "content": """
위 전체 내용을 종합하여 뉴스 리포팅을 해줘.
관련 링크를 추가해줘
---
'제목':
'본문':
        """
    }
])

for chunk in result:
    print(chunk.choices[0].delta.content, end='')

**제목**: SK텔레콤, 자체 개발한 AI 언어모델 탑재 'AI 고객센터' 오픈

**본문**: SK텔레콤이 통신에 특화된 자체 개발 언어모델인 텔코 거대언어모델(LLM)과 대규모 멀티모달 모델(LMM)을 활용한 고객센터 인공지능(AI) 상담 업무 지원 시스템의 베타 서비스를 성공적으로 운영했다고 18일 발표했다. 이 시스템은 지난달부터 한 달여간 시범 운영되었으며, SK텔레콤은 앤스로픽 등 해외 주요 LLM 업체들과 협력하여 AI 상담의 효율성을 높이고 있다. 이번 AI 고객센터는 국내 메이저 고객센터 중 최초로 도입된 것으로, 고객의 다양한 요구에 신속하게 대응할 수 있는 기능을 갖추고 있다.

이와 함께, SK텔레콤은 AI 기술을 통해 고객 서비스의 품질을 향상시키고, 고객의 편의성을 높이기 위한 다양한 노력을 지속하고 있다. 이러한 혁신적인 접근은 고객 경험을 개선하고, 통신 산업의 디지털 전환을 가속화하는 데 기여할 것으로 기대된다.

관련 링크:
- [네이버 뉴스 1](https://n.news.naver.com/mnews/article/001/0015051992?sid=105)
- [네이버 뉴스 2](https://n.news.naver.com/mnews/article/032/0003333070?sid=101)
- [한국일보](https://daily.hankooki.com/news/articleView.html?idxno=1148532)
- [화이트페이퍼](http://www.whitepaper.co.kr/news/articleView.html?idxno=240088)
- [인사이트코리아](https://www.insightkorea.co.kr/news/articleView.html?idxno=213933)None

이와 같은 스트리밍은 LLM 어플리케이션의 마지막 출력에서 사용하면    
사용자 경험을 향상시킬 수 있습니다.

<br><br><br>
## Tool(과거 Function Call) : 커스텀 함수 사용하기
만약 query를 명시적으로 지정하지 않고, 사용자의 질문에 따라 관련 query를 추출하고,  
이를 통해 검색하게 할 수도 있을까요?    

`Tool Calling` 를 추가하면 이를 수행할 수 있습니다.    

Web_Search라는 이름의 툴을 만들고, 이를 `get_html`과 연결하겠습니다.

-------


In [22]:
from pydantic import BaseModel, Field

class Web_Search(BaseModel):
    """query를 이용하여 뉴스 검색"""
    query: str= Field(description="""검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "개봉영화", "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"
        """)

tools = [openai.pydantic_function_tool(Web_Search)]
tools

[{'type': 'function',
  'function': {'name': 'Web_Search',
   'strict': True,
   'parameters': {'description': 'query를 이용하여 뉴스 검색',
    'properties': {'query': {'description': '검색 키워드\n규칙:\n1. 최대 2개 단어로 구성\n2. 불필요한 조사나 형용사 제외\n3. 핵심 명사만 포함\n\n예시:\n- (좋음) "개봉영화", "영화"\n- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"\n        ',
      'title': 'Query',
      'type': 'string'}},
    'required': ['query'],
    'title': 'Web_Search',
    'type': 'object',
    'additionalProperties': False},
   'description': 'query를 이용하여 뉴스 검색'}}]

Tool 정보가 포함된 gpt는 함수의 구조와 설명을 이용하여 어떤 함수를 써야 하는지를 판단합니다.

직접적으로 실행하는 것은 아니지만, 결과를 활용하면 해당 함수를 바로 사용할 수 있습니다.

In [23]:
def news_bot_v2(messages, stream=False, model = 'gpt-4o-mini'):

    # 사용할 툴 펑션의 목록과 설명을 리스트로 전달
    # LLM이 스스로 description, name, parameter의 값을 통해 판단
    class Web_Search(BaseModel):
        """query를 이용하여 뉴스 검색"""
        query: str= Field(description="""
검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"
""")

    tools = [openai.pydantic_function_tool(Web_Search)]

    response = client.chat.completions.create(
        model = model,
        messages = messages,

        tools = tools, # 사용할 툴 목록 전달
        tool_choice = 'auto',
        # 'none'이면 툴 사용하지 않음
        # 'required'면 무조건 툴 사용

        # parallel_tool_calls = False,
        # 툴 1회만 출력하기

        temperature= 0.1,
        max_tokens= 1024,

        stream = stream
        # stream = True 면 스트리밍, 아니면 기본 출력
    )

    # Streaming 여부에 따라 출력 다르게 하기
    if stream: return response

    return response.choices[0].message

In [24]:
# tool 사용 필요 없음
result = news_bot_v2([
    {
        "role": "user",
        "content": """안녕하세요! 오늘 날씨가 좋네요."""
    }
])

result

ChatCompletionMessage(content='안녕하세요! 날씨가 좋다니 기쁘네요. 오늘 어떤 계획이 있으신가요?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [26]:
# tool 사용 필요함

tool_call_result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요?
"""
    }
])

print(tool_call_result)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AA5rzStjlWaBwnq4qbr4IlMh', function=Function(arguments='{"query":"영화"}', name='Web_Search'), type='function')])


Tool Call 메시지는 content 대신 tool_calls로 전달됩니다.

In [27]:
tool_call_result.tool_calls

[ChatCompletionMessageToolCall(id='call_AA5rzStjlWaBwnq4qbr4IlMh', function=Function(arguments='{"query":"영화"}', name='Web_Search'), type='function')]

In [28]:
(tool_call_result.tool_calls[0].id,
tool_call_result.tool_calls[0].function.name,
tool_call_result.tool_calls[0].function.arguments)

('call_AA5rzStjlWaBwnq4qbr4IlMh', 'Web_Search', '{"query":"영화"}')

LLM이 스스로 판단하여, 실행할 함수의 정보를 `tool_calls`에 돌려줍니다.

tool_calls의 구성 요소는 다음과 같습니다.
- id: tool call의 id로, 해당 id에 실행 결과를 연결할 수 있습니다.
- function : arguments와 name을 통해 실행할 툴의 이름과 매개변수를 전달합니다.
<br><br>


tool_calls의 결과를 임의로 포함하여 전달해 보겠습니다.   
`tool`타입의 메시지로 전달해야 합니다.

In [31]:
result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요?
"""
    },
    tool_call_result, # tool_calls 메시지
    {
        "role": "tool",
        "content": """
청설, 보통의 가족, 아노라, 아마존 활명수, 베놈:라스트 댄스
""",
        "tool_call_id":tool_call_result.tool_calls[0].id
    }
])

result

ChatCompletionMessage(content='요즘 새로 개봉한 영화로는 다음과 같은 작품들이 있습니다:\n\n1. 청설\n2. 보통의 가족\n3. 아노라\n4. 아마존 활명수\n5. 베놈: 라스트 댄스\n\n관심 있는 영화가 있으면 더 알아보세요!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

이번에는 실제 함수를 실행하여 전달해 보겠습니다.

In [32]:
tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments

('Web_Search', '{"query":"영화"}')

함수명과 함수에 전달할 매개변수값이 주어지면, 아래와 같이 함수를 불러올 수 있습니다.

In [33]:
# 문자열을 Dict로 바꾸기
import json

example = '{"query":"LLM"}'
example_dict = json.loads(example)
# json으로 변환

print(example_dict)
print(type(example_dict))



{'query': 'LLM'}
<class 'dict'>


In [34]:
# 함수이름 문자열을 함수로 바꾸기

available_functions = {'Web_Search': get_news}

available_functions['Web_Search']

<function __main__.get_news(query)>

In [35]:
# Dictionary를 함수의 매개변수로 전달하기

def f(a,b):
  return a + b

example2 = {'a': 1, 'b': 2}

f(**example2)


3

In [36]:
available_functions['Web_Search'](**example_dict)

"제목: SKT AI 고객센터 오픈…&quot;통신 특화 <b>LLM</b> 적용&quot;\nURL: https://www.etoday.co.kr/news/view/2420398\n내용: 이투데이=안유리 기자 | 자체 개발한 Telco <b>LLM</b>, LMM 활용해 10월부터 고객센터 상담에 단계적 적용 문장형 AI 지식 검색, AI 서류 자동 처리, 상담 결과 자동 처리 SK텔레콤은 자체 개발한 Telco <b>LLM</b>(Large Language Model)과 LMM... \n---\n제목: SKT, AI 고객센터 운영…전용 <b>LLM</b>으로 상담 업무 효율화\nURL: https://n.news.naver.com/mnews/article/030/0003258363?sid=105\n내용: SK텔레콤이 자체 개발한 텔코(Telco) 거대언어모델(<b>LLM</b>)과 대형멀티모달모델(LMM)을 활용한 고객센터 인공지능... 연중 테스트와 피드백을 통한 강화 학습을 반복해 통신 서비스 전용 <b>LLM</b>, LMM 모델을 만들었다. 또 <b>LLM</b>이... \n---\n제목: SKT AI 고객센터 오픈…&quot;통신 특화 <b>LLM</b> 적용&quot;\nURL: http://www.metroseoul.co.kr/article/20241118500315\n내용: Telco <b>LLM</b>(Large Language Model)과 LMM(Large Multimodal Model)을 활용한 고객센터 AI 상담 업무 지원 시스템을 10월 중순부터 단계적으로 열고, 한 달여 간 베타서비스를 운영했다고 18일 밝혔다. SKT는 자체 개발한 Telco <b>LLM</b>/LMM을... \n---\n제목: SKT AI 고객센터 오픈...<b>LLM</b> · LMM 활용\nURL: https://www.digitaltoday.co.kr/news/articleView.html?idxno=541886\n내용: SK텔레콤 모델이 자체 개발한 T

위 과정을 조합하여 실행해 보겠습니다.

In [38]:
available_functions = {'Web_Search':get_news}

name, arguments = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments


result = news_bot_v2([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요? 오늘 날짜는 24년 11월 18일입니다.
"""
    },
    tool_call_result, # tool_calls 메시지
    {
        "role": "tool",
        "content": available_functions[name](**json.loads(arguments)),  
        # get_news(query = '영화')

        "tool_call_id":tool_call_result.tool_calls[0].id
    }
])

result.content

"현재 개봉 예정인 영화 중 하나는 **'하얼빈'**입니다. 이 영화는 12월 25일 크리스마스에 개봉할 예정이며, 배우 현빈과 이동욱이 출연합니다. 영화는 1909년을 배경으로 하며, 독립운동과 관련된 이야기를 다루고 있습니다. \n\n더 자세한 내용은 [여기](https://www.viva100.com/article/20241118500216)에서 확인하실 수 있습니다."

스트리밍 기능을 추가하고, 전체 과정을 함수로 자동화합니다.:

In [39]:
def news_qa(prompt):

    print('Prompt:',prompt)
    # 출력 결과를 함수와 연결하기 위한 dictionary
    available_functions = {'Web_Search':get_news}

    tool_call_result = news_bot_v2([
        {
            "role": "user",
            "content": prompt
        }
    ])
    # 프롬프트를 받아 Web_Search를 수행할지 결정합니다.
    # 단순 대화라면 tool call을 생성하지 않습니다.

    print('---')
    print('News_Bot: Call ', end='')

    if tool_call_result.tool_calls: # tool_calls가 존재하면:

        name, arguments = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments

        print(name,arguments)

        # get_news(query)
        result = available_functions[name](**json.loads(arguments))
        # 함수 선택, 매개변수 전달
        print('---')
        print('News_Bot:',end='')

        response = news_bot_v2(
        [
        {
            'role':"system",
            'content':'사용자의 질문과 이를 답변하기 위한 정보가 주어집니다. 이를 바탕으로 답변하세요.'
        },

        {
            "role": "user",
            "content": prompt
        },
        tool_call_result, # tool_calls 메시지
        {
            "role": "tool",
            "content": result,
            "tool_call_id":tool_call_result.tool_calls[0].id
        }],
        stream=True
        )
        for chunk in response:
            print(chunk.choices[0].delta.content, end='')
    else:
        print('Nothing')
        print(tool_call_result.content)

# example
# "오늘 가장 큰 뉴스는 뭐야?"
# """오픈AI는 요즘 새로운 소식 있어?"""
prompt = """넷플릭스 신작 추천해줘."""

news_qa(prompt)

Prompt: 넷플릭스 신작 추천해줘.
---
News_Bot: Call Web_Search {"query":"넷플릭스 신작"}
---
News_Bot:최근 넷플릭스에서 공개된 신작 중 추천할 만한 작품은 다음과 같습니다:

1. **좀비버스: 뉴 블러드 (Zombieverse: New Blood)** - K-좀비 예능의 신기원을 연 이 프로그램은 11월 19일에 공개됩니다. 좀비 코미디와 버라이어티 요소가 결합된 재미있는 콘텐츠입니다.

2. **가석방 심사관 이한신** - 11월 18일에 공개되는 이 드라마는 긴장감 넘치는 스토리라인을 가지고 있어 기대를 모으고 있습니다.

3. **이별, 그 뒤에도** - 이 드라마는 사랑과 이별을 다룬 이야기로, 11월 18일에 공개됩니다.

이 외에도 다양한 장르의 신작들이 있으니, 취향에 맞는 작품을 찾아보세요!None

In [40]:
news_qa("회사 가기 싫어.")

Prompt: 회사 가기 싫어.
---
News_Bot: Call Nothing
회사가 가기 싫은 기분이 드는 건 자연스러운 일이에요. 어떤 이유로 그런 기분이 드는지 이야기해볼까요? 스트레스, 업무량, 동료와의 관계 등 여러 가지 이유가 있을 수 있어요.


# [심화] Multiple Tool Call
때로는, 하나의 메시지에서 여러 개의 Tool Call을 수행해야 하기도 합니다.   


반복문을 통해, Tool Call의 목록을 한꺼번에 전달합니다.

In [41]:
def news_qa_v2(prompt, model = 'gpt-4o'):

    print('Prompt:',prompt)
    # 출력 결과를 함수와 연결하기 위한 dictionary
    available_functions = {'Web_Search':get_news}

    tool_call_result = news_bot_v2([
        {
            "role": "user",
            "content": prompt
        },
    ])
    # 프롬프트를 받아 Web_Search를 수행할지 결정합니다.
    # 단순 대화라면 tool call을 생성하지 않습니다.

    print('---')
    print('News_Bot: Call ', end='')

    if tool_call_result.tool_calls: # tool_calls가 존재하면:



        ### Tool Message 복수 입력

        tool_messages=[]
        for tool_call in tool_call_result.tool_calls:
            name, arguments = tool_call.function.name, tool_call.function.arguments
            print(name,arguments)

            # get_news(query)
            result = available_functions[name](**json.loads(arguments))
            # 함수 선택, 매개변수 전달
            print('---')
            print('News_Bot:',end='')
            tool_messages.append(
                {"role": "tool","content": result,"tool_call_id":tool_call.id}
            )
        ####



        response = news_bot_v2(
        [
        {
            'role':"system",
            'content':'사용자의 질문과 이를 답변하기 위한 정보가 tool 실행 결과를 통해 주어집니다. 전체 내용을 최대한 자세히 읽고, 상세하게 답변하세요.'
        },

        {
            "role": "user",
            "content": prompt
        },
        tool_call_result] + tool_messages,
        stream=True,
        model = model
        )
        for chunk in response:
            print(chunk.choices[0].delta.content, end='')
    else:
        print('Nothing')
        print(tool_call_result.content)


prompt = """넷플릭스 왓챠 디즈니플러스 웨이브 신작 추천해줘."""

news_qa_v2(prompt)

Prompt: 넷플릭스 왓챠 디즈니플러스 웨이브 신작 추천해줘.
---
News_Bot: Call Web_Search {"query": "넷플릭스 신작"}
---
News_Bot:Web_Search {"query": "왓챠 신작"}
---
News_Bot:Web_Search {"query": "디즈니플러스 신작"}
---
News_Bot:Web_Search {"query": "웨이브 신작"}
---
News_Bot:다양한 OTT 플랫폼에서의 신작 추천을 위해 최근 뉴스 기사를 참고했습니다. 아래는 넷플릭스, 왓챠, 디즈니플러스, 웨이브에서 추천할 만한 신작입니다.

### 넷플릭스
1. **신작: 'Mr. 포트너'**
   - **장르**: 드라마
   - **내용**: 이 드라마는 주인공이 다양한 사건을 통해 성장하는 이야기를 담고 있습니다. 11월 14일에 공개됩니다.
   - **링크**: [기사 링크](https://m.entertain.naver.com/article/311/0001797192)

2. **신작: '좀비버스: 뉴 블러드'**
   - **장르**: 리얼리티, 좀비
   - **내용**: K-좀비를 테마로 한 리얼리티 쇼로, 참가자들이 좀비 상황에서 생존하는 과정을 그립니다. 11월 19일에 공개됩니다.
   - **링크**: [기사 링크](https://sports.hankooki.com/news/articleView.html?idxno=6881142)

### 왓챠
1. **신작: '신작'**
   - **장르**: 다양한 장르
   - **내용**: 왓챠에서 다양한 신작들이 공개되고 있으며, 특히 한국 드라마와 영화가 주목받고 있습니다. 구체적인 제목은 기사에서 확인할 수 있습니다.
   - **링크**: [기사 링크](https://m.entertain.naver.com/article/609/0000922147)

### 디즈니플러스
1. **신작: '가디언즈 오브 갤럭시 Vol. 3'**
   - **장르**: 액션, 어드벤처


In [42]:
news_qa_v2(prompt, model='gpt-4o')

Prompt: 넷플릭스 왓챠 디즈니플러스 웨이브 신작 추천해줘.
---
News_Bot: Call Web_Search {"query": "넷플릭스 신작"}
---
News_Bot:Web_Search {"query": "왓챠 신작"}
---
News_Bot:Web_Search {"query": "디즈니플러스 신작"}
---
News_Bot:Web_Search {"query": "웨이브 신작"}
---
News_Bot:최근 각 스트리밍 플랫폼에서 공개된 신작들을 소개합니다.

### 넷플릭스
1. **좀비버스: 뉴 블러드** - K-좀비 예능으로, 좀비와의 생존을 다루는 콘텐츠입니다.
2. **미키 17** - 북미 극장에서 개봉 예정인 영화로, 넷플릭스 시리즈 '성난 사람들'과 함께 주목받고 있습니다.
3. **사흘** - 박신양 주연의 드라마로, 복잡한 인간관계를 다루고 있습니다.

### 왓챠
1. **유마 카운티의 끝에서: 주유소 살인사건** - 범죄 스릴러 영화로, 긴장감 넘치는 스토리를 제공합니다.
2. **러브 라이즈 블리딩** - 로맨스 드라마로, 사랑과 갈등을 그립니다.
3. **개소리** - 코미디 드라마로, 일상 속의 유머를 담고 있습니다.

### 디즈니플러스
1. **강남 비-사이드** - 한국 드라마로, 강남의 화려한 삶을 배경으로 한 이야기를 다룹니다.
2. **조명가게** - 미스터리 드라마로, 조명가게에서 벌어지는 사건들을 중심으로 전개됩니다.
3. **왕과 사는 남자** - 역사 드라마로, 조선시대를 배경으로 한 이야기입니다.

### 웨이브
1. **미안하다, 사랑한다 2024** - 클래식 드라마의 리메이크로, 현대적인 감각을 더했습니다.
2. **여왕벌 게임** - 서바이벌 게임을 소재로 한 드라마로, 긴장감 넘치는 전개가 특징입니다.
3. **내 이름은 김삼순 2024** - 인기 드라마의 리메이크로, 새로운 캐스팅과 함께 돌아왔습니다.

각 플랫폼에서 다양한 장르의 신작들이 공개되고 있으니, 관심 있는 작품을 찾아보세요!N